In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200622.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,6/17/2020 19:26:45,Colossus: The Forbin Project,Computer Chess,Arcade,The Lawnmower Man,Johnny Mnemonic,Electric Dreams,Spike Lee Joints,"Oh shit, its Satan!",Rapper in major role,Films adapted from the stage,Dance Battle,Makeover Montage,No Theme/Free-for-all
1,6/17/2020 19:41:28,Arcade,Electric Dreams,The Lawnmower Man,Computer Chess,Johnny Mnemonic,Colossus: The Forbin Project,Rapper in major role,Dance Battle,Spike Lee Joints,"Oh shit, its Satan!",Films adapted from the stage,Makeover Montage,No Theme/Free-for-all
2,6/17/2020 20:22:27,Computer Chess,Colossus: The Forbin Project,Electric Dreams,Arcade,The Lawnmower Man,NaN,No Theme/Free-for-all,Films adapted from the stage,"Oh shit, its Satan!",Dance Battle,Rapper in major role,Makeover Montage,Spike Lee Joints
3,6/17/2020 20:24:24,Colossus: The Forbin Project,Computer Chess,The Lawnmower Man,Electric Dreams,Johnny Mnemonic,Arcade,"Oh shit, its Satan!",Rapper in major role,Makeover Montage,Films adapted from the stage,Dance Battle,Spike Lee Joints,No Theme/Free-for-all
4,6/17/2020 20:38:12,Colossus: The Forbin Project,Arcade,Computer Chess,Electric Dreams,NaN,NaN,Spike Lee Joints,Rapper in major role,Dance Battle,Films adapted from the stage,Makeover Montage,"Oh shit, its Satan!",NaN
5,6/18/2020 4:44:55,Electric Dreams,Arcade,Johnny Mnemonic,Colossus: The Forbin Project,Computer Chess,The Lawnmower Man,Makeover Montage,Rapper in major role,Dance Battle,Films adapted from the stage,"Oh shit, its Satan!",Spike Lee Joints,No Theme/Free-for-all
6,6/20/2020 11:13:29,Arcade,Computer Chess,Johnny Mnemonic,Colossus: The Forbin Project,Electric Dreams,The Lawnmower Man,"Oh shit, its Satan!",Films adapted from the stage,Rapper in major role,Spike Lee Joints,Makeover Montage,Dance Battle,No Theme/Free-for-all
7,6/20/2020 11:16:26,Arcade,Electric Dreams,Computer Chess,Colossus: The Forbin Project,Johnny Mnemonic,The Lawnmower Man,Makeover Montage,"Oh shit, its Satan!",Spike Lee Joints,No Theme/Free-for-all,Rapper in major role,Films adapted from the stage,Dance Battle


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Colossus: The Forbin Project', 'Computer Chess', 'Arcade',
       'The Lawnmower Man', 'Johnny Mnemonic', 'Electric Dreams'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Colossus: The Forbin Project': <Candidate('Colossus: The Forbin Project')>,
 'Computer Chess': <Candidate('Computer Chess')>,
 'Arcade': <Candidate('Arcade')>,
 'The Lawnmower Man': <Candidate('The Lawnmower Man')>,
 'Johnny Mnemonic': <Candidate('Johnny Mnemonic')>,
 'Electric Dreams': <Candidate('Electric Dreams')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Colossus: The Forbin Project, Computer Chess, Arcade, The Lawnmower Man, Johnny Mnemonic, Electric Dreams)>,
 <Ballot(Arcade, Electric Dreams, The Lawnmower Man, Computer Chess, Johnny Mnemonic, Colossus: The Forbin Project)>,
 <Ballot(Computer Chess, Colossus: The Forbin Project, Electric Dreams, Arcade, The Lawnmower Man)>,
 <Ballot(Colossus: The Forbin Project, Computer Chess, The Lawnmower Man, Electric Dreams, Johnny Mnemonic, Arcade)>,
 <Ballot(Colossus: The Forbin Project, Arcade, Computer Chess, Electric Dreams)>,
 <Ballot(Electric Dreams, Arcade, Johnny Mnemonic, Colossus: The Forbin Project, Computer Chess, The Lawnmower Man)>,
 <Ballot(Arcade, Computer Chess, Johnny Mnemonic, Colossus: The Forbin Project, Electric Dreams, The Lawnmower Man)>,
 <Ballot(Arcade, Electric Dreams, Computer Chess, Colossus: The Forbin Project, Johnny Mnemonic, The Lawnmower Man)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                       Votes  Status
----------------------------  -------  --------
Arcade                              3  Hopeful
Colossus: The Forbin Project        3  Hopeful
Computer Chess                      1  Rejected
Electric Dreams                     1  Rejected
The Lawnmower Man                   0  Rejected
Johnny Mnemonic                     0  Rejected

FINAL RESULT
Candidate                       Votes  Status
----------------------------  -------  --------
Colossus: The Forbin Project        4  Elected
Arcade                              4  Rejected
Computer Chess                      0  Rejected
Electric Dreams                     0  Rejected
The Lawnmower Man                   0  Rejected
Johnny Mnemonic                     0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                       Votes  Status
----------------------------  -------  --------
Oh shit, its Satan!                 2  Hopeful
Spike Lee Joints                    2  Hopeful
Makeover Montage                    2  Hopeful
Rapper in major role                1  Rejected
No Theme/Free-for-all               1  Rejected
Films adapted from the stage        0  Rejected
Dance Battle                        0  Rejected

ROUND 2
Candidate                       Votes  Status
----------------------------  -------  --------
Oh shit, its Satan!                 3  Hopeful
Spike Lee Joints                    3  Hopeful
Makeover Montage                    2  Rejected
Rapper in major role                0  Rejected
No Theme/Free-for-all               0  Rejected
Films adapted from the stage        0  Rejected
Dance Battle                        0  Rejected

FINAL RESULT
Candidate                       Votes  Status
----------------------------  -------  --------
Oh shit, its Satan! 